# AI 批量视频生成平台 - Cloudflare Tunnel 版本

使用自定义域名访问服务

## 使用方法

### 首次使用:
1. 运行 Cell 0-2: 安装依赖
2. 运行 Cell 3: 登录 Cloudflare (浏览器授权)
3. 运行 Cell 4: 创建 Tunnel (记录名称)
4. 运行 Cell 5: 配置 DNS
5. 运行 Cell 6: 下载备份文件
6. 修改 Cell 7 的配置
7. 运行 Cell 7: 启动服务

### 后续使用:
运行 Cell 0-2, 然后运行 Cell 7 即可

In [ ]:
# Cell 0: 克隆项目
# !git clone https://github.com/ge0rgeguo/ecommerce_video_generation_ai.git 2>/dev/null || echo "✅ 项目已存在"
# %cd ecommerce_video_generation_ai/ai_batch_generation_website

In [ ]:
# Cell 1: 安装 Python 依赖
# !pip install -q fastapi uvicorn sqlalchemy bcrypt cryptography pillow nest_asyncio requests
# print("✅ Python 依赖安装完成")

✅ Python 依赖安装完成


In [ ]:
# Cell 2: 安装 cloudflared
# ⚠️ 根据环境选择运行对应的命令

# ============================================================
# 🐧 Colab/Linux 环境 (Google Colab 运行这个)
# ============================================================
# !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
# !dpkg -i cloudflared-linux-amd64.deb 2>/dev/null
# !cloudflared --version
# print("✅ cloudflared 安装完成 (Linux)")

# ============================================================
# 🍎 本地 Mac 环境 (本地测试运行这个)
# ============================================================
# 方法 1: 使用 Homebrew (推荐)
# !brew install cloudflared

# 方法 2: 手动安装 (Apple Silicon)
# !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-darwin-arm64.tgz
# !tar -xzf cloudflared-darwin-arm64.tgz
# !chmod +x cloudflared
# !sudo mv cloudflared /usr/local/bin/
# !rm cloudflared-darwin-arm64.tgz
# !cloudflared --version
# print("✅ cloudflared 安装完成 (Mac)")


Password:sudo: a password is required
zsh:1: command not found: cloudflared
✅ cloudflared 安装完成


---
## 首次配置 (只需运行一次)
---

In [17]:
# Cell 3: 登录 Cloudflare (首次需要)
# ⚠️ 会输出 URL,在浏览器中打开并授权
!cloudflared tunnel login

A browser window should have opened at the following URL:

https://dash.cloudflare.com/argotunnel?aud=&callback=https%3A%2F%2Flogin.cloudflareaccess.org%2FbRMIZ5MXbFtzdwipRor85NHT8jkluEPPw8W-q3-n4R8%3D

If the browser failed to open, please visit the URL above directly in your browser.
2025-10-21T03:28:21Z INF You have successfully logged in.
If you wish to copy your credentials to a server, they have been saved to:
/Users/zyma/.cloudflared/cert.pem



In [18]:
# Cell 4: 创建 Tunnel (首次需要)
import uuid
tunnel_name = f"colab-webapp-{uuid.uuid4().hex[:8]}"
!cloudflared tunnel create {tunnel_name}
print(f"\n✅ Tunnel 名称: {tunnel_name}")
print("⚠️ 请记住这个名称,复制到 Cell 7")

Tunnel credentials written to /Users/zyma/.cloudflared/681cc3ac-997f-441b-8482-138252909ed4.json. cloudflared chose this file based on where your origin certificate was found. Keep this file secret. To revoke these credentials, delete the tunnel.

Created tunnel colab-webapp-172bb0f0 with id 681cc3ac-997f-441b-8482-138252909ed4

✅ Tunnel 名称: colab-webapp-172bb0f0
⚠️ 请记住这个名称,复制到 Cell 7


In [ ]:
# Cell 5: 配置 DNS (首次需要)
# ⚠️ 修改这两个变量
TUNNEL_NAME = "colab-webapp-172bb0f0"  # 替换为 Cell 4 的名称
YOUR_DOMAIN = "zuoshipin.net"        # 替换为你的域名

!cloudflared tunnel route dns {TUNNEL_NAME} {YOUR_DOMAIN}
print(f"✅ DNS 已配置: {YOUR_DOMAIN}")

2025-10-21T03:30:26Z INF Added CNAME zuoshipin.net which will route to this tunnel tunnelID=22e49757-f9a7-4a4e-aa59-1f3b9785f46b
✅ DNS 已配置: zuoshipin.net


In [10]:
# Cell 6: 下载认证文件备份 (首次建议运行)
from google.colab import files
import os, glob

if os.path.exists('/root/.cloudflared/cert.pem'):
    files.download('/root/.cloudflared/cert.pem')
    print("✅ cert.pem 已下载")

for f in glob.glob('/root/.cloudflared/*.json'):
    files.download(f)
    print(f"✅ {os.path.basename(f)} 已下载")

print("\n💾 请保存这些文件,下次使用时运行 Cell 8 上传")

ModuleNotFoundError: No module named 'google.colab'

---
## 启动服务 (每次运行)
---

In [1]:
!python -m venv colab_website_venv
!source colab_website_venv/bin/activate

In [ ]:
# Cell 7: 启动服务
# ⚠️ 修改以下配置

import os, sys, nest_asyncio, uvicorn, subprocess, threading, time
from pathlib import Path

# ==================== 配置 ====================
TUNNEL_NAME = "colab-webapp-172bb0f0"  # 替换为 Cell 4 的名称
YOUR_DOMAIN = "zuoshipin.net"        # 替换为你的域名  
YUNWU_API_KEY = "sk-LRZGHDhEJzL2DKPJ4DHtz3tfCfU8VQzymAtKvuI9WyspsVFh"           # Yunwu API 密钥
# =============================================

os.environ["YUNWU_API_KEY"] = YUNWU_API_KEY
os.environ["PUBLIC_BASE_URL"] = f"https://{YOUR_DOMAIN}"
if "DISABLE_BACKGROUND" in os.environ:
    del os.environ["DISABLE_BACKGROUND"]

# 根据环境选择正确的目录
import platform
import glob
if platform.system() == "Linux":
    # Colab 环境
    cloudflared_dir = Path("/root/.cloudflared")
else:
    # 本地 Mac/Windows
    cloudflared_dir = Path.home() / ".cloudflared"

cloudflared_dir.mkdir(parents=True, exist_ok=True)

# 查找 credentials 文件 (cloudflared 使用 tunnel ID 作为文件名,而不是 tunnel name)
# 文件名格式: <uuid>.json
creds_files = list(cloudflared_dir.glob("*.json"))

if not creds_files:
    print(f"❌ 错误: 在 {cloudflared_dir} 中找不到任何 .json 认证文件")
    print(f"💡 解决方案:")
    print(f"   1. 如果是首次配置,运行 Cell 3-6")
    print(f"   2. 如果已有备份,运行 Cell 8 上传认证文件")
    print(f"   3. 或者重新运行 'cloudflared tunnel login' 和 'cloudflared tunnel create'")
    raise FileNotFoundError(f"No credentials file found in {cloudflared_dir}")

# 使用第一个找到的 .json 文件
creds_file = creds_files[0]
print(f"✅ 找到认证文件: {creds_file.name}")

config_file = cloudflared_dir / "config.yml"
with open(config_file, "w") as f:
    f.write(f"""tunnel: {TUNNEL_NAME}
credentials-file: {str(creds_file)}
ingress:
  - hostname: {YOUR_DOMAIN}
    service: http://localhost:8888
  - service: http_status:404
""")

print("="*60)
print(f"🌐 域名: https://{YOUR_DOMAIN}")
print(f"👤 登录: admin / admin000")
print("="*60)

# 后台启动 Cloudflare Tunnel
threading.Thread(
    target=lambda: subprocess.run(["cloudflared", "tunnel", "run", TUNNEL_NAME]),
    daemon=True
).start()

sys.path.append(os.getcwd())
nest_asyncio.apply()
time.sleep(3)

print("🚀 启动服务...\n")
config = uvicorn.Config("server.app:app", host="0.0.0.0", port=8888, log_level="info")
server = uvicorn.Server(config)
await server.serve()

✅ 找到认证文件: 681cc3ac-997f-441b-8482-138252909ed4.json
🌐 域名: https://zuoshipin.net
👤 登录: admin / admin000


2025-10-21T03:32:17Z INF Starting tunnel tunnelID=681cc3ac-997f-441b-8482-138252909ed4
2025-10-21T03:32:17Z INF Version 2025.10.0 (Checksum 33c410f017b0d83b8d9c818cedf5cacd9cb4b1ccfc95af961a94850f0abf3696)
2025-10-21T03:32:17Z INF GOOS: darwin, GOVersion: go1.24.2, GoArch: arm64
2025-10-21T03:32:17Z INF Settings: map[cred-file:/Users/zyma/.cloudflared/681cc3ac-997f-441b-8482-138252909ed4.json credentials-file:/Users/zyma/.cloudflared/681cc3ac-997f-441b-8482-138252909ed4.json]
2025-10-21T03:32:17Z INF Autoupdate frequency is set autoupdateFreq=86400000


🚀 启动服务...



/opt/anaconda3/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.generics:GenericModel` has been moved to `pydantic.BaseModel`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
INFO:     Started server process [64543]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8888 (Press CTRL+C to quit)


[startup] Background executor starting...
[startup] Cleanup loop started. Public dir: /Users/zyma/Projects/ecommerce_video_generation_ai/ai_batch_generation_website/public


2025-10-21T03:32:27Z ERR Failed to fetch features, default to disable error="lookup cfd-features.argotunnel.com on 198.18.0.2:53: dial udp 198.18.0.2:53: i/o timeout"
2025-10-21T03:32:27Z INF Generated Connector ID: 70363800-3cb1-4f08-a530-51493deac89c
2025-10-21T03:32:47Z WRN Unable to lookup protocol percentage.
2025-10-21T03:32:47Z INF Initial protocol http2
2025-10-21T03:32:47Z INF ICMP proxy will use 192.168.2.52 as source for IPv4
2025-10-21T03:32:47Z INF ICMP proxy will use fe80::cd0:8187:cc00:3864 in zone en0 as source for IPv6
2025-10-21T03:32:47Z INF Created ICMP proxy listening on 192.168.2.52:0
2025-10-21T03:33:00Z INF ICMP proxy will use 192.168.2.52 as source for IPv4
2025-10-21T03:33:00Z INF ICMP proxy will use fe80::cd0:8187:cc00:3864 in zone en0 as source for IPv6
2025-10-21T03:33:00Z INF Starting metrics server on 127.0.0.1:20241/metrics
2025-10-21T03:33:11Z INF You requested 4 HA connections but I can give you at most 2.
2025-10-21T03:33:13Z INF Registered tunnel con

2025-10-21T03:33:46Z INF Unregistered tunnel connection connIndex=1 event=0 ip=198.18.0.57
2025-10-21T03:33:46Z INF Unregistered tunnel connection connIndex=0 event=0 ip=198.18.0.58
2025-10-21T03:33:46Z INF Retrying connection in up to 1s connIndex=1 event=0 ip=198.18.0.57
2025-10-21T03:33:46Z ERR Connection terminated connIndex=1
2025-10-21T03:33:46Z INF Retrying connection in up to 1s connIndex=0 event=0 ip=198.18.0.58
2025-10-21T03:33:46Z ERR Connection terminated connIndex=0
2025-10-21T03:33:46Z ERR no more connections active and exiting
2025-10-21T03:33:46Z INF Tunnel server stopped
2025-10-21T03:33:46Z INF icmp router terminated error="read udp 192.168.2.52:0: raw-read udp 192.168.2.52:0: use of closed network connection"
2025-10-21T03:33:46Z INF Metrics server stopped


---
## 恢复认证文件 (可选)
---

In [ ]:
# Cell 8: 上传认证文件
from google.colab import files
import shutil, os

uploaded = files.upload()
os.makedirs('/root/.cloudflared', exist_ok=True)

for filename in uploaded.keys():
    shutil.move(filename, f'/root/.cloudflared/{filename}')
    print(f"✅ {filename} 已恢复")

print("\n✅ 现在可以运行 Cell 7 启动服务")